In [2]:
import pandas as pd

# reading csv file 
df = pd.read_csv("Sample_Req.csv")

In [1]:
import os

os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
)

In [5]:
from langchain.prompts import PromptTemplate

template = '''
You are given a pandas data frame. You need to convert the data frame into JSON object.
1. Return a valid JSON. No preamble. 
2. The JSON object's keys should correspond to the column headers of the data frame, and each row should become a JSON object with its respective column values

Here is the data frame on which you need to perform this task:  
{data_frame}
'''

pt = PromptTemplate.from_template(template)

In [6]:
from langchain_core.output_parsers import JsonOutputParser

chain = pt | llm | JsonOutputParser()
response = chain.invoke(input={"data_frame": df})

In [23]:
# from langchain_core.exceptions import OutputParserException
# from langchain_core.output_parsers import JsonOutputParser

# try:
#     json_parser = JsonOutputParser()
#     res = json_parser.parse(response.content)
# except OutputParserException:
#     raise OutputParserException("Context too big. Unable to parse jobs.")

In [7]:
from langchain_core.documents import Document

documents = [(Document(page_content=str(item), metadata={"Requirement ID": item["id"]})) for item in response]

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
import uuid
from langchain_chroma import Chroma

ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

client = Chroma(collection_name=unique_collection_name, embedding_function=embeddings_model)

# Add documents
client.add_documents(documents=documents, ids = ids)

['c02b06e2-aa44-4e44-ad18-982763b3e783',
 '6a136966-94bc-480c-87a8-4065b06823cd',
 '77d03aaf-0265-4e59-afd2-2f0b7221fd31',
 '7a0e00b8-a4e8-4621-8015-05eefd006300',
 '8e864ee9-8f9c-41db-917d-84f2f8f8c63e',
 '365d4888-e79d-4e73-81c4-42dca11b2bbf',
 'cb2c73bb-2640-4bd1-bd4d-a28603a25e03',
 'fe7a9ea0-859f-4b94-b1de-c65fda25866d',
 '80e5ea6f-0465-4196-a766-3facb907b05a',
 '5b6b510e-032d-4bea-bf5a-6081eb89bf02',
 'fad327e3-2366-4ccb-b439-c4f1889535cf',
 'fae5e69c-5f62-4deb-a702-0f36e10ec773',
 '1e864570-5f2d-4002-bf69-b194d6c37abf']

In [15]:
# resp = db.get(include=["embeddings"])

# print(resp)

In [26]:
all_documents = client.get()["documents"]

type(all_documents)

list

In [10]:
from langchain_core.tools import tool
from langchain.agents import Tool

# Define the tool for writing test cases
tools = [
    Tool(
        name="write_test_case",
        description="Generate structured test cases for input data",
        func=lambda input: writetestcasetool(
        input["args"]["primary_text"], input["args"]["verification_criteria"], input["args"]["id"]
        )
    )
]

def writetestcasetool(primary_text,verification_criteria,id_val):
    """Usefull to generate test cases for primary text for requirement"""
    test_case_id = "TC_" + str(hash(primary_text))[:6]  # Generate a unique test case ID
    test_case = {
        "Test Case ID": test_case_id,
        "Requirement Coverd": id_val,
        "Test Case Title": f"Test for {primary_text[:30]}...",  # Shortened primary text as title
        "Objective": f"Validate that the system meets the requirement: {primary_text}",
        "Pre-conditions": "Define the system's initial state required for the test.",
        "Steps": [
            f"Step 1: {verification_criteria.split(',')[0].strip()}",
            "Step 2: Additional steps based on requirement."
        ],
        "Expected Results": (
            f"Each test step is performed correctly, and the outcomes match the requirement's intent: {primary_text}. "
            f"Confirm proper system behavior as per verification criteria: {verification_criteria}."
        )
    }
    return json.dumps(test_case, indent=2)

In [12]:
from crewai import Agent, Task, Crew, Process, LLM

In [13]:
model = LLM(model="groq/llama-3.3-70b-versatile")

In [30]:
TC_Generation_Agent = Agent(
    role = "Test Case Generator",
    goal = """Create test cases which will validate / verify the functionality mentioned in the input requirements""",
    backstory = """You are a test case generation assistant who will create test cases for system requirements of automotive applications. The requirements are stored in a vector store and provided to you in JSON object format.
        You should generate the valid test cases for all the individual documents in vector store.""",
    verbose=True,
    allow_delegation=True,
#    memory=True,
    llm=model,
    tools=tools
)

task = Task(
    description = f"Understand the given content and generate relevant test cases to validate each requirement",
    agent = TC_Generation_Agent,
    expected_output="Generate test cases which includes clear Test Case ID(interger number followed by TC), Requirement covered('id' attribute from input), objective, Pre-condition, steps for validation, expected results, and any necessary edge cases or exceptions."
)

crew = Crew(
    agent = [TC_Generation_Agent],
    tasks = [task],
    verbose = True,
    process=Process.sequential,
#    memory=True
)

output = crew.kickoff(inputs=response[1])

# Agent: Test Case Generator
## Task: Understand the given content and generate relevant test cases to validate each requirement
 

I encountered an error while trying to use the tool. This was the error: 'args'.
 Tool write_test_case accepts these inputs: Tool Name: write_test_case
Tool Arguments: {'input': {'description': '', 'type': 'Any'}}
Tool Description: Generate structured test cases for input data



# Agent: Test Case Generator
## Thought: To approach this task, I first need to understand the requirements provided in the JSON object format. Since I don't have the specifics of the requirements yet, I'll need to generalize my approach based on the information given. The goal is to generate test cases that validate the functionality mentioned in these requirements. 
Given that I have access to the "write_test_case" tool, I should utilize it to create structured test cases for the input data. However, without the actual requirements, I'll describe a generic approach to creating t

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jcw5kw32e2btxg9qkcncq2tt` on : Limit 100000, Used 100143, Requested 873. Please try again in 14m38.388s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Test Case Generator
## Task: Understand the given content and generate relevant test cases to validate each requirement


ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jcw5kw32e2btxg9qkcncq2tt` on : Limit 100000, Used 100139, Requested 466. Please try again in 8m42.849999999s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Test Case Generator
## Task: Understand the given content and generate relevant test cases to validate each requirement


ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jcw5kw32e2btxg9qkcncq2tt` on : Limit 100000, Used 100134, Requested 466. Please try again in 8m38.650999999s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jcw5kw32e2btxg9qkcncq2tt` on : Limit 100000, Used 100134, Requested 466. Please try again in 8m38.650999999s. Visit https://console.groq.com/docs/rate-limits for more information.","type":"","code":"rate_limit_exceeded"}}


In [28]:
type(response[1])

dict